apply the bilateral blur then sustract it from the original image then add the result to the original image

In [2]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

hdr_img = cv.imread("image/image.tif", -1)
cv.imshow('TiffExemple',hdr_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:

img8 = (hdr_img/256).astype('uint8')
print(img8.dtype)
blur = cv.bilateralFilter(img8,9,75,75)
 
cv.imshow('TiffExemple',(img8-blur)+img8)
cv.waitKey(0)
cv.destroyAllWindows()


uint8
